<a href="https://colab.research.google.com/github/LordHandLee/HonCSC499_detect_falls_soccer/blob/main/CNN_base_fallen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import numpy as np
import tensorflow as tf
from tensorflow import keras
import glob

import matplotlib.pyplot as plt
import cv2
from skimage import data,io
import matplotlib.image as mpimg

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.layers.experimental import preprocessing
#from autokeras.utils import data_utils

from sklearn.model_selection import train_test_split
import pathlib

In [2]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
data_dir = "/content/drive/MyDrive/honor_thesis/image_depo/"
# data_dir = pathlib.Path(data_dir).with_suffix('')
# for i in data_dir.glob("*"):
#   print(i)
datagen = ImageDataGenerator(rescale=1/255.0, validation_split=0.3)
width = 150
height = 150
trainDatagen = datagen.flow_from_directory(directory=data_dir, target_size=(width, height),
                                           class_mode='binary',
                                           batch_size=16,
                                           subset='training')
valDatagen = datagen.flow_from_directory(directory=data_dir, target_size=(width, height),
                                           class_mode='binary',
                                           batch_size=16,
                                           subset='validation')

Found 611 images belonging to 2 classes.
Found 260 images belonging to 2 classes.


In [4]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=30)


In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))



In [6]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False), optimizer=keras.optimizers.Adam(learning_rate=0.01), metrics=[keras.metrics.BinaryAccuracy()])

history = model.fit(trainDatagen, steps_per_epoch = len(trainDatagen),
                    epochs=500, validation_data = valDatagen,
                    validation_steps = len(valDatagen),
                    callbacks=[early_stop])

Epoch 1/500
39/39 [==============================] - 361s 9s/step - loss: 3.4990 - binary_accuracy: 0.5565 - val_loss: 0.6776 - val_binary_accuracy: 0.5692
Epoch 2/500
39/39 [==============================] - 3s 75ms/step - loss: 0.6600 - binary_accuracy: 0.5974 - val_loss: 0.6557 - val_binary_accuracy: 0.6308
Epoch 3/500
39/39 [==============================] - 3s 76ms/step - loss: 0.6599 - binary_accuracy: 0.5941 - val_loss: 0.6790 - val_binary_accuracy: 0.5269
Epoch 4/500
39/39 [==============================] - 3s 78ms/step - loss: 0.6006 - binary_accuracy: 0.6563 - val_loss: 0.8067 - val_binary_accuracy: 0.5962
Epoch 5/500
39/39 [==============================] - 3s 75ms/step - loss: 0.6201 - binary_accuracy: 0.6432 - val_loss: 0.6281 - val_binary_accuracy: 0.7154
Epoch 6/500
39/39 [==============================] - 3s 73ms/step - loss: 0.5657 - binary_accuracy: 0.7152 - val_loss: 0.6302 - val_binary_accuracy: 0.7462
Epoch 7/500
39/39 [==============================] - 3s 78ms/ste

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 73984)             0